# Introdução
- Notebook utilizado para tratarmos inicialmente a conversa, trocando os usuários do chat por nomes genéricos.

In [ ]:
!pip install emoji

In [ ]:
import re
import regex
import pandas as pd
import numpy as np
import emoji
import random

Foi criado um dicionário para armazenar o nome dos usuários como chave, e o nome anonimizado para ser substituído.

In [ ]:
author_dict = {}
pattern_phone = re.compile('@[0-9]{12}')

In [ ]:
def startsWithDateAndTime(s):
    pattern = '^([0-2][0-9]|(3)[0-1])(\/)(((0)[0-9])|((1)[0-2]))(\/)([2][0][0-2][0-9]+) ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -' 
    result = re.match(pattern, s)
    if result:
        return True
    return False

In [ ]:
def FindAuthor(s):
  s=s.split(":")
  if len(s)==2:
    return True
  else:
    return False

In [ ]:
def recheckName(tempText):
  for key in author_dict:
    if key in tempText:
      tempText = tempText.replace(key, author_dict[key])
  return tempText

Nesta função, abrimos dois arquivos, o do chat original, e o novo arquivo que estará com os nomes anônimos.

A ideia utilizada para anonimizar os nomes foi utilizar de um array auxiliar com 1000 posições (valor arbritrário), e a cada vez que um nome é encontrado no chat, é verificado se aquele nome está no dicionário. Como de início o dicionário é vázio, o nome será adicionado como chave, e um valor é anexado a ele: "User" concatenado com um valor aleatório do array auxiliar. Quando o valor é concatenado excluímos esse valor do array, de forma que cada valor seja único.

Quando um nome é encontrado, fazemos um replace, baseado no dicionário criado.

A ideia desta função é passar linha por linha da conversa do whatsapp, identificando padrões. No chat do whatsapp, fica fácil identificar que sempre inicia com data e hora, depois vem "-" seguido do usuário do grupo, precedido do ":". Com esse padrão definido foi isolado o nome do usuário.

Para a exclusão do número foi criado uma expressão regular para identificar um número na conversa, seja ele autor da mensagem, seja uma citação no conteúdo da mensagem. De forma que se o padrão for identificado, ele é substituído por "XXX-hidden number-XXX". 

Por fim, fizemos um função para verificar se nas mensagens os autores das mensagens citavam outros participantes, e essa função substitui esses casos utilizando o dicionário criado.


In [ ]:
inputFile = 'chat_whatsapp.txt'
outputFile = open("chat_whatsapp_anonymate.txt", "a+") 
randomArray = list(range(1,1000))
with open(inputFile, encoding="utf-8") as fp:
    fp.readline() # Skipping first line of the file because contains information related to something about end-to-end encryption
    while True:
        line = fp.readline() 
        if not line: 
            break
        line = line.strip() 
        if startsWithDateAndTime(line):
            splitLine = line.split(' - ') 
            dateTime = splitLine[0]
            message = ' '.join(splitLine[1:])
            if FindAuthor(message): 
                splitMessage = message.split(': ')
                if not (splitMessage[0] in author_dict):
                  number = random.choice(randomArray)
                  author_dict[splitMessage[0]] = "User" + str(number)
                  randomArray.remove(number)                
                line = line.replace(splitMessage[0], author_dict[splitMessage[0]])
        line = re.sub(pattern_phone, "XXX-hidden number-XXX", line)          
        line = recheckName(line)
        outputFile.write(line)
        outputFile.write("\n")

outputFile.close()